In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="D90rVo7ahGyBHqpqtBIH")
project = rf.workspace("indian-institute-of-information-technology-sricity").project("wall-crack-detection-own")
dataset = project.version(3).download("folder")

In [ ]:
import os
def walkthrough_dir(dir_path):
  for dpath, dname, filename in os.walk(dir_path):
    print(f'There are {len(dname)} directories and {len(filename)} files in {dpath}.')

In [ ]:
train_dir = '/kaggle/working/wall-crack-detection-own-3/train'
valid_dir = '/kaggle/working/wall-crack-detection-own-3/valid'
test_dir = '/kaggle/working/wall-crack-detection-own-3/test'

In [ ]:
walkthrough_dir('/kaggle/working/wall-crack-detection-own-3')

# **Getting Data Ready**

In [ ]:
import tensorflow as tf
IMG_SIZE = (640,640)
train_data = tf.keras.preprocessing.image_dataset_from_directory(train_dir,
                                                                 label_mode='categorical',
                                                                 image_size=IMG_SIZE)
valid_data = tf.keras.preprocessing.image_dataset_from_directory(valid_dir,
                                                                 label_mode='categorical',
                                                                 image_size=IMG_SIZE,
                                                                 shuffle=False)
test_data = tf.keras.preprocessing.image_dataset_from_directory(test_dir,
                                                                label_mode='categorical',
                                                                image_size=IMG_SIZE,
                                                                shuffle=False)

# **Checkpoint**
---
A **checkpoint** is an intermediate dump of a model's entire internal state (its weights, current learning rate, etc.) so that the framework can resume the training from this point whenever desired.

In [ ]:
checkpoint_path = 'fruits_classification_model_checkpoint'
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                         save_weights_only=True,
                                                         monitor='val_accuracy',
                                                         save_best_only=True)

# **Data Augmentation**
---
**Data augmentation** is the process of transforming images to create new ones, for training machine learning models.

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.models import Sequential

data_augmentation = Sequential([
    preprocessing.RandomFlip('horizontal'),
    preprocessing.RandomRotation(0.2),
    preprocessing.RandomHeight(0.2),
    preprocessing.RandomWidth(0.2),
    preprocessing.RandomZoom(0.2),
    preprocessing.Rescaling(1/255.)
],name='data_augmenation')

# **Model Building**
---
We will be using the Keras **Functional API** which is a way to create models that are more flexible than the **tf.keras.Sequential API**. The functional API can handle models with non-linear topology, shared layers, and even multiple inputs or outputs.

The main idea is that a deep learning model is usually a **directed acyclic graph (DAG)** of layers. So the functional API is a way to build graphs of layers.

We will be using 2 pretrained models here. So basically this will be **Transfer Learning**.The reuse of a pre-trained model on a new problem is known as transfer learning in machine learning. A machine uses the knowledge learned from a prior assignment to increase prediction about a new task in transfer learning.
1. EfficientNetB0
2. MobileNetV2

**MobileNetV2**

In [ ]:
base_model_1 = tf.keras.applications.MobileNetV2(include_top=False)
base_model_1.trainable = False

inputs = layers.Input(shape=(IMG_SIZE[0], IMG_SIZE[1], 3), name="input_layer")
x = data_augmentation(inputs)
x = base_model_1(x, training=False)
x = layers.GlobalAveragePooling2D(name="global_average_pooling")(x)
outputs = layers.Dense(len(train_data.class_names), activation="softmax", name="output_layer")(x)
mobilenet_model = tf.keras.Model(inputs, outputs)

In [ ]:
mobilenet_model.summary()

In [ ]:
mobilenet_model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])
mobilenet_model_hist = mobilenet_model.fit(train_data,
                 epochs=10,
                 validation_data=valid_data,
                 #validation_steps=int(0.15 * len(valid_data)),
                 callbacks=[checkpoint_callback])

In [ ]:
mobilenet_model_loss, mobilenet_model_acc = mobilenet_model.evaluate(test_data)

In [ ]:
# Specify the file path where you want to save your model
model_save_path = '/kaggle/working/models/mobilenet_model.h5'

# Save the model to the specified file
mobilenet_model.save(model_save_path)

print(f"Model saved to {model_save_path}")

In [ ]:
# print("Training Loss    : {:.4} | Baseline : {:.4}".format(mobilenet_model_loss, efficient_model_loss))
# print("Training Accuracy: {:.4}% | Baseline : {:.4}%".format(mobilenet_model_acc*100, efficient_model_acc*100))

In [ ]:
import matplotlib.pyplot as plt

# Assuming you have already trained your model and saved the training history in mobilenet_model_hist.

# Plot training & validation loss values
plt.plot(mobilenet_model_hist.history['loss'])
plt.plot(mobilenet_model_hist.history['val_loss'])
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'], loc='upper right')
plt.show()

# Plot training & validation accuracy values
plt.plot(mobilenet_model_hist.history['accuracy'])
plt.plot(mobilenet_model_hist.history['val_accuracy'])
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'], loc='lower right')
plt.show()

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score

def save_confusion_matrix(model, data, title, filename):
    predictions = model.predict(data)
    predicted_labels = np.argmax(predictions, axis=1)
    
    # Assuming data is a tf.data.Dataset, extract labels
    labels = np.concatenate([y for x, y in data], axis=0)
    true_labels = np.argmax(labels, axis=1)
    
    accuracy = accuracy_score(true_labels, predicted_labels)
    cm = confusion_matrix(true_labels, predicted_labels)
    
    num_classes = len(cm)
    
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=range(num_classes), yticklabels=range(num_classes))
    plt.xlabel('Predicted Labels')
    plt.ylabel('True Labels')
    plt.title(f'{title} (Accuracy: {accuracy:.2f})')
    plt.savefig(filename)
    plt.show()

# Save confusion matrices for train, validation, and test data
# save_confusion_matrix(mobilenet_model, train_data, "Train Data Confusion Matrix", "train_confusion_matrix.png")
save_confusion_matrix(mobilenet_model, valid_data, "Validation Data Confusion Matrix", "valid_confusion_matrix.png")
save_confusion_matrix(mobilenet_model, test_data, "Test Data Confusion Matrix", "test_confusion_matrix.png")
